In [1]:
!pip install pyspark
!pip install findspark

Could not fetch URL https://pypi.org/simple/pyspark/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pyspark/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)'))) - skipping


ERROR: Could not find a version that satisfies the requirement pyspark (from versions: none)
ERROR: No matching distribution found for pyspark


In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [3]:
scala_version = '2.12'  # your scala version
spark_version = '3.5.0' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [24]:
# Usage of KafkaConsumer class
bootstrap_servers = 'localhost:9092'
topic_name = 'PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7'

kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", bootstrap_servers)\
                .option("subscribe", topic_name)\
                .option("startingOffsets", "earliest").load()

In [8]:
selected_columns = ["rating", "reviewerName", "reviewText", "categories", "gPlusPlaceId", "unixReviewTime", "reviewTime", "gPlusUserId"]

In [26]:
from time import sleep
from IPython.display import display, clear_output
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7").getOrCreate()

# Định nghĩa schema cho dữ liệu JSON
json_schema = StructType([
    StructField("rating", DoubleType(), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("gPlusPlaceId", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("gPlusUserId", StringType(), True)
])

# Đọc dữ liệu từ Kafka topic và chuyển đổi thành DataFrame
kafka_stream_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap_servers) \
    .option("subscribe", topic_name) \
    .load()

kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", bootstrap_servers)\
                .option("subscribe", topic_name)\
                .option("startingOffsets", "earliest").load()

# Chuyển đổi giá trị từ JSON string sang struct với schema đã định nghĩa
json_stream_df = kafkaDf.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", json_schema).alias("data")) \
    .select("data.*")

x = 0
while True:
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        x += 1

        newest = json_stream_df.tail(30)  # Chọn 30 dòng mới nhất

        # Hiển thị từng dòng một
        last_30_rows_df = spark.createDataFrame(newest)
        last_30_rows_df.show(truncate=False)

        sleep(5)
        clear_output(wait=True)

    except KeyboardInterrupt:
        print("break")
        break

print("Live view ended...")


Showing live view refreshed every 5 seconds
Seconds passed: 30
+------+----------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------

In [27]:
json_stream_df.show(100)

+------+----------------+-------------------------------------+--------------------+--------------------+--------------+------------+--------------------+
|rating|    reviewerName|                           reviewText|          categories|        gPlusPlaceId|unixReviewTime|  reviewTime|         gPlusUserId|
+------+----------------+-------------------------------------+--------------------+--------------------+--------------+------------+--------------------+
|   3.0|          an lam|                    Chất lượng tạm ổn| ["Giải Trí - Café"]|10810331438000420...|    1372686659| Jul 1, 2013|10000001081715426...|
|   5.0|    HALİL TURGUT|                 Wc si temiz duzen...| ["Turkish Cuisine"]|10219412824160874...|    1342870724|Jul 21, 2012|10000001350028553...|
|   5.0|      森田さとこ|何回も私は予定に休みがセルバに行っ...|["Fishing","Pond ...|10140985882817540...|    1390653513|Jan 25, 2014|10000002133684886...|
|   5.0|      森田さとこ|今度は予定に休みが登米市に行きたい...|          ["Museum"]|10147717750015851...|    13891